In [1]:
import numpy as np
import sympy as sp
import itertools as it
import string
import pickle
from tqdm import tqdm
import time

In [2]:
from sympy.physics.quantum import HermitianOperator, Dagger

In [3]:
part_diz = { 'A' : 3, 'B' : 2}
parties = list(part_diz.keys())
inputs = [part_diz[i] for i in parties]
max_in=[str(i) for i in range(max(inputs))]
operators=[]
for i in list(part_diz.keys()):
    tmp=[]
    for l in range(part_diz[i]):
        tmp.append(str(l))
        print(str(l))
    operators+=[''.join(k) for k in it.product(i, tmp, repeat = 1)]

level = 1

0
1
2
0
1


In [4]:
operators

['A0', 'A1', 'A2', 'B0', 'B1']

In [5]:
monomial_set = []

for i in range(1,level+1):   
    
    lev = [''.join(k) for k in it.product(operators, repeat = i)]
    
    #commutatività parties
    for n in range(i-1):
        for k in range(len(lev)):
            for l1 in parties: 
                for l2 in parties: 
                    if l2<l1:
                        for n1 in max_in:
                            for n2 in max_in:
                                if l1+n1+l2+n2 in lev[k]:
                                    lev[k] = lev[k].replace(l1+n1+l2+n2,l2+n2+l1+n1)
    if i == 1:
        operators = lev.copy()
    #unitarietà operatori
    for p in range(len(lev)):      
        for m in range(1,int(i/2)+1):
            #s = [''.join(k) for k in it.product(operators, repeat = m)]
            while any(el+el in lev[p] for el in operators):
                for el in operators:
                    lev[p] = lev[p].replace(el+el, '')
                        
    
        
    
    monomial_set += lev
monomial_set+=['']
monomial_set = sorted(sorted(list(set(monomial_set))), key=len)
print(len(monomial_set))

6


In [6]:
monomial_set

['', 'A0', 'A1', 'A2', 'B0', 'B1']

In [7]:
n=2
rev_mon_set = []
for l in range(len(monomial_set)):
    lista = [monomial_set[l][i:i+n] for i in range(0, len(monomial_set[l]), n)]
    lista.reverse()
    rev_mon_set.append(''.join(lista))
print(rev_mon_set, monomial_set)

['', 'A0', 'A1', 'A2', 'B0', 'B1'] ['', 'A0', 'A1', 'A2', 'B0', 'B1']


In [8]:
gamma_mat = []
for i in range(len(monomial_set)):
    gamma_row = []
    for j in range(i, len(rev_mon_set)):
        gamma_row.append(rev_mon_set[j]+monomial_set[i])
    gamma_mat.append(gamma_row)

In [9]:
for i in tqdm(range(len(gamma_mat))):
    #time.sleep(0.01)
    for j in range(len(gamma_mat[i])):
        #print(i,j)
#        while any(l2<l1 and l1+n1+l2+n2 in gamma_mat[i][j] for l1 in originals for l2 in originals for n1 in inputs for n2 in inputs):
#            for l1 in originals:
#                for l2 in originals:
#                    if l2<l1:
#                        for n1 in inputs:
#                            for n2 in inputs:
#                                gamma_mat[i][j] = gamma_mat[i][j].replace(l1+n1+l2+n2,l2+n2+l1+n1)
        for length in range(int(len(gamma_mat[i][j])/2)-1):
            for l1 in parties: 
                for l2 in parties: 
                    if l2<l1:
                        for n1 in max_in:
                            for n2 in max_in:
                                if l1+n1+l2+n2 in gamma_mat[i][j]:
                                    gamma_mat[i][j] = gamma_mat[i][j].replace(l1+n1+l2+n2,l2+n2+l1+n1)
        
              
    
        for m in range(1,int(len(gamma_mat[i][j])/2)+1):
            #s = [''.join(k) for k in it.product(operators, repeat = m)]
            while any(el+el in gamma_mat[i][j] for el in operators):
                for el in operators:
                    #print(gamma_mat[i][j], el+el, i,j)
                    gamma_mat[i][j] = gamma_mat[i][j].replace(el+el, '')
                    #print(gamma_mat[i][j])
                    #print('***')

100%|██████████████████████████████████████████████████████████████████████████████████| 6/6 [00:00<00:00, 6006.16it/s]


In [10]:
monomial_index = {}
for i in range(len(gamma_mat)):
    for j in range(len(gamma_mat[i])):
        monomial_index[gamma_mat[i][j]] = []

for i in range(len(gamma_mat)):
    for j in range(len(gamma_mat[i])):
        monomial_index[gamma_mat[i][j]].append([i,i+j])

In [11]:
diz_constraints = { list(monomial_index.keys())[i] : i  for i in range(len(list(monomial_index.keys())))}

In [12]:
index_keys = list(monomial_index.keys())

In [13]:
index_keys

['',
 'A0',
 'A1',
 'A2',
 'B0',
 'B1',
 'A1A0',
 'A2A0',
 'A0B0',
 'A0B1',
 'A2A1',
 'A1B0',
 'A1B1',
 'A2B0',
 'A2B1',
 'B1B0']

In [14]:
A = np.zeros([len(index_keys),len(monomial_set),len(monomial_set)])
for i in range(len(index_keys)):
    for j in range(len(monomial_index[index_keys[i]])):
        A[i][monomial_index[index_keys[i]][j][0]][monomial_index[index_keys[i]][j][1]] = 1.

In [15]:
for i in range(len(index_keys)):
    A[i]=A[i]+A[i].T

In [16]:
filename = f'matrici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'wb') as f:
    pickle.dump(A, f)
filename = f'indici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'wb') as g:
    pickle.dump(index_keys, g)

In [17]:
filename = f'indici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'rb') as g:
    indici_cons = pickle.load(g)
filename = f'matrici_dei_constraints_instrumental_lev_{level}'
with open(filename, 'rb') as f:
    dizionario_cons = pickle.load(f)

In [18]:
len(indici_cons)

16

In [19]:
diz = { index_keys[i] : dizionario_cons[i]  for i in range(len(index_keys)) }

In [20]:
np.all(np.abs(diz[index_keys[i]]-diz[index_keys[i]].T) < 0.00001)

True

In [21]:
for i in diz.keys():
    if np.all(diz[i]==0):
        print('damn')

In [22]:
gamma_mat

[['', 'A0', 'A1', 'A2', 'B0', 'B1'],
 ['', 'A1A0', 'A2A0', 'A0B0', 'A0B1'],
 ['', 'A2A1', 'A1B0', 'A1B1'],
 ['', 'A2B0', 'A2B1'],
 ['', 'B1B0'],
 ['']]